Цель данного домашнего задания — познакомить вас с концепцией и реализацией RAG в LLM, а также развить навыки интеграции механизмов поиска с языковыми моделями. Вы научитесь извлекать данные из специализированных источников, использовать их для поддержки генеративного процесса и оценивать качество полученных результатов.

# Устанавливаем зависимости

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install datasets sentence_transformers==3.3.1 trl

# Текст на чанки - 5 баллов

Для наших заданий будем использовать датасет rag-dataset-12000, в котором есть вопросы, ответы и большие контексты.

In [ ]:
from datasets import load_dataset
wikiq = load_dataset('neural-bridge/rag-dataset-12000')

print(len(wikiq['train']))
print(wikiq['train'][0]['question'], '\n')
print(wikiq['train'][0]['answer'], '\n')
print(wikiq['train'][0]['context'], '\n')

Для налаживания процессов возьме 100 самых длинных текстов

In [ ]:
k_longest = 100
train_data = sorted(wikiq['train'], key=lambda w: len(w["context"]))[-k_longest:]

train_full_docs = [elem['context'] for elem in train_data]
train_queries = [elem['question'] for elem in train_data]

Реализуйте класс Chunker, в котором метод split_text_to_chunks разбивает входный текст на чанки с перекрытием и возвращает список полученных чанков, и метод get_chunked_list, которые по списку текстов возвращает список чанков из этих текстов согласно методу split_text_to_chunks. Размер чанка и перекрытие измеряется в кол-ве слов, то есть chunk_words = 10 означает, что чанк состоит из 10 слов (слова - сущности, которые получаются после простого сплита строки по одному пробелу " "), перекрытие 3 означает, что если взять два соседний чанка, то 3 последних слова первого являются 3мя первыми словами второго чанка.

In [ ]:
class Chunker:
    def __init__(self, chunk_words: int = 100, overlap: int = 30):
        # your code here

    def split_text_to_chunks(self, text: str) -> list[str]:
        # your code here

    def get_chunked_list(self, texts: list[str]) -> list[str]:
        # your code here

In [ ]:
chunker = Chunker(chunk_words=100, overlap=30)
train_chunked_docs = chunker.get_chunked_list(train_full_docs)
print(train_chunked_docs, '\n')
print(train_chunked_docs[0], '\n')
print(train_chunked_docs[1])

In [ ]:
print(len(train_chunked_docs))

# Векторный поиск - 10 баллов

Построить простой векторный поиск на основе энкодерной модели modernbert-embed-base. Модель устроена так, что эмбеддинги запросов (всегда должны начинаться с префикса "search_query: ") всегда близки в векторном пространстве эмбеддингам похожих/релевантных документов (их текст всегда должен начинаться на "search_document: ")

In [ ]:
from sentence_transformers import SentenceTransformer

device = "cpu"

model = SentenceTransformer("nomic-ai/modernbert-embed-base").to(device)

query_embeddings = model.encode([
    "search_query: What is TSNE?",
    "search_query: Who is Laurens van der Maaten?",
])
doc_embeddings = model.encode([
    "search_document: TSNE is a dimensionality reduction algorithm created by Laurens van Der Maaten",
])
print(query_embeddings.shape, doc_embeddings.shape)
# (2, 768) (1, 768)

similarities = model.similarity(query_embeddings, doc_embeddings)
print(similarities)
# tensor([[0.7214],
#         [0.3260]])

In [ ]:
import numpy as np

class Encoder:
    def __init__(self, embed_model_name: str = "nomic-ai/modernbert-embed-base",
                 device: str = "cuda"):
        self.embed_model = SentenceTransformer(embed_model_name).to(device)

    def encode_query(self, texts: list[str]) -> np.ndarray:
        # тут энкодим запросы, не забываем про префикс

    def encode_docs(self, texts: list[str]) -> np.ndarray:
        # тут энкодим документы, не забываем про префикс

    def similarity(self, *args, **kwargs):
        # тут считаем косинусную близость


class VectorSearchEngine:
    def __init__(self,
                 init_base: list[str],
                 encoder: Encoder,
                 ):
        # тут строим векторный индекс по исходной базе init_base с помощью энкодера

    def insert_doc(self, doc: str) -> None:
        # тут добавляем в векторный индекс и в список документов новый документ

    def get_k_most_similar(self, query: str, k: int) -> tuple:
        # тут пытаемся оптимально найти k ближайших для query документов, вернуть список текстов этих доков в порядке убывания близости,
        # а также соответствующими им похожести

In [ ]:
encoder = Encoder(device="cuda")

In [ ]:
vse = VectorSearchEngine(train_chunked_docs[:3], encoder)

In [ ]:
vse.get_k_most_similar(train_queries[0], k=2)

In [ ]:
train_queries[0]

# Генератор - 5 баллов

Реализуйте метод generate генерации языковой модели, который по списку запросов выдает список текстовый ответов. **kwargs должны пойти как аргументы в model.generate при генерации.

In [ ]:
device = "cuda"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

class Generator:
    def __init__(self, model_name: str = "Qwen/Qwen2-0.5B", device: str = "cuda" ):
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            device_map="auto"#=device
        ).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def generate(self, inputs: list[str],
                 **kwargs) -> str:
        #  your code here

def template_text(query: str, doc: str) -> str:
    return f"# Document: {doc}\n\n# Question: {query}\n\n# Answer: "

In [ ]:
generator = Generator(device=device)

In [ ]:
doc = "Turunturundia is a captivating and enigmatic country located in a secluded part of the world, yet to be discovered by adventurous travelers. The landscape of Turunturundia is a harmonious blend of expansive emerald plains, lush forests, and towering mountains capped with eternal snow. Its people are known for their warm hospitality and celebrate a rich tapestry of cultural traditions passed down through generations. Turunturundia's history is steeped in legends and folklore, with ancient ruins and artifacts indicating a civilization that valued art, philosophy, and nature. The capital of Turunturundia's is Turuncity. The country is also home to several unique species of flora and fauna found nowhere else on Earth, making it a treasure trove for botanists and ecologists alike. Despite its modest size, Turunturundia's spirit and charm leave a lasting impression on all who have the privilege of exploring its wonders."
query = "what is the capital of turunturundia?"
templated_example = template_text(query, doc)

generator.generate([templated_example], max_new_tokens=512)

# Генерация гипотез - 5 баллов

функция template_text готовит вход в нужном формате. Реализуйте функцию create_hypotheses, которая по заданному входу генерирует n_candidates + 1 генераций:
* n_candidates с помощью семплинга с заданной температурой
* 1 кандидат - greedy генерация (без семлинга, макс вероятность каждого токена)
и возвращает в формате: (гриди генерация, список семлпинг генераций)

Семлпинг и его параметры можно найти в документации

In [ ]:
def create_hypotheses(templated_example: str,
                      generator: Generator,
                      temperature: float = 0.5,
                      max_new_tokens: int = 512,
                      n_candidates: int = 10) -> (str, list[str]):
    # your code here

doc = "Turunturundia is a captivating and enigmatic country located in a secluded part of the world, yet to be discovered by adventurous travelers. The landscape of Turunturundia is a harmonious blend of expansive emerald plains, lush forests, and towering mountains capped with eternal snow. Its people are known for their warm hospitality and celebrate a rich tapestry of cultural traditions passed down through generations. Turunturundia's history is steeped in legends and folklore, with ancient ruins and artifacts indicating a civilization that valued art, philosophy, and nature. The capital of Turunturundia's is Turuncity. The country is also home to several unique species of flora and fauna found nowhere else on Earth, making it a treasure trove for botanists and ecologists alike. Despite its modest size, Turunturundia's spirit and charm leave a lasting impression on all who have the privilege of exploring its wonders."
query = "what is the capital of turunturundia?"
templated_example = template_text(query, doc)

greedy_example, hypotheses = create_hypotheses(templated_example, generator)
greedy_example, hypotheses

# Reward Model

Возьмем из открытого доступа реворд-модель, которая обучалась понимать, какой из 2 ответов лучше, и будем использовать ее pointwise - для оценки одного ответа. Будем скорить всех сгенерированных на один запрос кандидатов и брать максимальный по скору. А пока просто посмотрим на ее ранжирующие свойства для кандидатов.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

class RewardModel:
    def __init__(self,
                 model_name: str = "OpenAssistant/reward-model-deberta-v3-large-v2",
                 device: str = "cuda"
                 ):
        rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(model_name), AutoTokenizer.from_pretrained(model_name)
        self.rank_model = rank_model.to(device)
        self.tokenizer = tokenizer

    def get_score(self, q: str, ans: str) -> float:
        inputs = self.tokenizer(q, ans, return_tensors='pt').to(self.rank_model.device)
        score = self.rank_model(**inputs).logits[0].cpu().detach()
        return score


reward_model = RewardModel()

In [ ]:
doc = "Turunturundia is a captivating and enigmatic country located in a secluded part of the world, yet to be discovered by adventurous travelers. The landscape of Turunturundia is a harmonious blend of expansive emerald plains, lush forests, and towering mountains capped with eternal snow. Its people are known for their warm hospitality and celebrate a rich tapestry of cultural traditions passed down through generations. Turunturundia's history is steeped in legends and folklore, with ancient ruins and artifacts indicating a civilization that valued art, philosophy, and nature. The capital of Turunturundia's is Turuncity. The country is also home to several unique species of flora and fauna found nowhere else on Earth, making it a treasure trove for botanists and ecologists alike. Despite its modest size, Turunturundia's spirit and charm leave a lasting impression on all who have the privilege of exploring its wonders."
query = "what is the capital of turunturundia?"
templated_example = template_text(query, doc)

greedy_example, hypotheses = create_hypotheses(templated_example, generator)

ranks = []
for ans in hypotheses:
    score = reward_model.get_score(query, ans)
    ranks.append((score, query, ans))

sorted_ranks = sorted(ranks, key=lambda w: w[0])

for elem in sorted_ranks:
    print(elem, '\n')

# Всё вместе и RS - 10 баллов

In [ ]:
k_longest = 100
train_data = sorted(wikiq['train'], key=lambda w: len(w["context"]))[-k_longest:]

train_full_docs = [elem['context'] for elem in train_data]
train_queries = [elem['question'] for elem in train_data]

chunker = Chunker(chunk_words=100, overlap=30)
train_chunked_docs = chunker.get_chunked_list(train_full_docs)

encoder = Encoder(device="cpu")
vse = VectorSearchEngine(train_chunked_docs, encoder)

generator = Generator()
reward_model = RewardModel()

На основе решений выше соберите pandas датафрейм, в котором каждая строка будет состоять из запроса, контекста, одной из генераций (гриди или семплинг), типа генерации (гриди или семплинг) и значения реворда для данного ответа на данный запрос.
Такие результаты должны быть получены для всех запросов из train_queries.
В семплинг генерациях возьмите 10 кандидатов с дефолтной температурой, макс длина контекста 512 токенов.

In [ ]:
import pandas as pd

dct = {
    "query" : [],
    "ctx": [],
    "generation": [],
    "type": [],
    "score": []
}

# your code here

df = pd.DataFrame(dct)
df['idx'] = np.arange(df.shape[0])
df.head(3)

In [ ]:
print(df[df["type"] == "greedy"]["score"].mean())
# среднее значение ревордов генератор без дообучения. После дообучения оно аналогичная статистика должна стать больше

Напишите логику для Rejection Sampling: сохраните в best_score_df только те строки из df, в которых для запроса и контекста выбрана гипотеза с максимальным скором реворда. Посчитайте среднее значение реворда в такой выборке и сравните со среднем значениям только по гриди генерациям.

In [ ]:
rows = []
# your code here

best_score_df = pd.DataFrame(rows)

print(best_score_df["score"].mean())
best_score_df.head(3)

# Обучение - 10 баллов

Попробуем улучшить наш генератор с помощью данных, полученных на предыдущем шаге. Весь пайплайн выглядит так:
1. Мы сгенерировали ряд гипотез
2. Мы оценили ответы с помощью reward модели
3. Мы берем лучшие ответы для того, чтобы обучить на них модель

Таким образом мы получим новый датасет, на котором сможем обучиться.


In [ ]:
# создаем датасет в нужном формате
def formatting_prompts_func(example):
    output_texts = []
    return template_text(example["query"], example["ctx"]) + example["generation"].strip()

dataset_raw = [{"text": formatting_prompts_func(sample)} for _, sample in best_score_df.iterrows()]
print(*dataset_raw[:3], sep="\n--------\n")

Дальше написан код обучения, подробнее, что как работает мы разберем на следующей лекции! Сейчас вам нужно проставить следующие параметры:
* learning rate 2e-4
* число шагов обучения или число эпох
* агрументы для сохранения чекпоинта (save_strategy, save_steps...)

Все аргументы описаны в [документации](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
from datasets import Dataset
from trl import SFTTrainer, SFTConfig
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

model = generator.model.float()
tokenizer = generator.tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dataset = Dataset.from_list(dataset_raw)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

args = SFTConfig(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    
    
    # Заполните этот блок аргументов
    # num_train_epochs = 1, # 1 эпоха = 1 полный проход по данным
    # max_steps = 60, # сколько шагов обучения сделать
    save_strategy=...,
    save_interval=...,
    learning_rate = # ваш код здесь,
    ############
    
    fp16 = True,
    logging_steps = 1,
    optim = "adamw_hf",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
    dataset_text_field = "text",
    max_seq_length = 512,
    dataset_num_proc = 2,
    packing = False,
    report_to=None,
    load_best_model_at_end=True,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = args
)

trainer_stats = trainer.train()


После того, как вы обучили свою модель, давайте проверим наш пайплайн еще раз, посчитаем среднюю награду наших генераций, она должна была увеличиться

In [ ]:
# ваш код здесь

# Расширение запроса - 5 баллов

Зачастую пользователи предоставляют нам неподробный запрос и хочется его переписать или расширить для лучшего поиска по базе данных. Мы рассмотрим самый простой вариант расширения запроса - давайте добавим в запрос синонимов к каждому слову! Для этого нам поможет wordnet!

In [ ]:
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

synsets = wordnet.synsets('dog')
for sn in synsets[:3]:
    for lemma in sn.lemmas()[:4]:
        print(lemma.name().replace("_", " "))

Ваша задача дописать функцию expand_query: она должна проходиться по всем словам из текста и добавлять по одному синониму на каждое слово в запрос. Посмотрите, как поменяется близость между расширенным query и documents по сравнению с обычнм query и documents!

In [ ]:
def expand_query(query: str) -> str:
    words = query.split()
    # ваш код здесь
    pass

documents = [
    "The Eiffel Tower is a landmark in Paris, France.",
    "Paris is the capital of France and known for its art, fashion, and culture.",
    "France has a rich history, including revolutions and world wars.",
    "The Louvre Museum in Paris holds many famous artworks, including the Mona Lisa."
]
query = "Paris landmarks"
model = encoder.embed_model
# ваш код здесь